# Word Embedding 이란
**단어를 밀집 벡터(dense vector)의 형태로 표현하는 방법을 워드 임베딩(word embedding)** 이라고 한다. 워드 임베딩 결과로 나온 밀집 벡터를 **임베딩 벡터(embedding vector)** 라고 한다.
워드 임베딩 방법론에는 통계적 방식과 딥러닝 방식이 있다. **Word2Vec은 대표적인 딥러닝 방식의 워드 임베딩이다.**
- 워드 임베딩은 단어를 실수 벡터 공간으로 옮기는 방법으로 단어들의 의미의 유사성이 반영되도록 하는 방법이다.
    - 의미의 유사성이 반영된다는 의미: **비슷한 의미를 가지는 단어는 비슷한 값들로 구성**되도록 한다.


> ### 희소(sparse) 표현
> 대표적인 희소표현은 one-hot encoding이 있다. 모든 단어를 포괄하는 Vector의 차원에서 index 역할을 하는 하나의 숫자만 1이며 나머지는 모두 0으로 표현한다. word 개수가 vector의 차원이 되므로 공간적 낭비가 크다. 또한 각 단어의 의미를 표현하지 못한다.

> ### 밀집(dense) 표현
>밀집 표현은 단어를 사용자가 설정한 차원의 벡터로 표현한다. 단어를 표현하는 벡터는 실수 값들을 가지며 **단어를 표현**하는 일련의 숫자들이 들어간다. 

(참고) 앞에 DTM 같은 거는 문장에서 특징을 추출했다면(빈도수), word embedding은 단어 안에서 특징을 추출

## 분산표현(Distributed Representation)
- 분포 가설(Distribution Hypothesis)을 기반으로 하는 이론으로 **비슷한 위치에 등장하는 단어들은 비슷한 의미를 가진다** 라는 것이다. 즉 비슷한 의미를 가진 단어들은 그 주변 단어 분포도 비슷하다.
- 예
    - 나는 **밥을** 먹는다.
    - 나는 **빵을** 먹는다.
    - 나는 **국수를** 먹는다.
    - 위의 `밥`, `빵`, `국수` 은 모두 `나는` 과 `먹는다` 사이에 있다. 그럼 `밥, 빵, 국수`는 비슷한 의미의 단어다.
- word2vec, fastext 등이 **분산표현**을 기반으로 만들어진 모델이다.

# Word2Vec
- 딥러닝 기반 word embedding 방식. 딥러닝 모델은 `입력층-은닉층-출력층` 의 단순한 구조로 되어있다.
    - Fully connected layer로 이뤄진 단순한 딥러닝 모델이다.
    - 학습이 끝나면 Fully connected layer의 weights 를 embedding vector로 사용한다.
  
## CBOW, Skip-gram
- Word2Vec 학습 방식은 **CBOW(Continuous Bag of Words)와 Skip-gram 방식** 으로 나뉜다.    
- 두 방식 모두 분포가설에 입각해 주변단어와 중심단어간의 관계를 학습하여 embedding vector를 만든다. 
- CBOW 모델은 주변 단어들로부터 중심 단어를 예측하는 방식이다.
    -  나는 어제 ___ 함께 학교에 갔다.
- Skip-gram 모델은 중심 단어로부터 주변 단어들을 예측하는 방식이다.
    -   ____ 밥을 _____
 
![cbow_skipgram_outline](figures/word2vec_cbow_skipgram_outline.png)
> 중심단어: **target 단어**, 기준 단어<br>
> 주변단어: **target 주위**의 단어.  **context(맥락), window**라고 한다.<br>
> 주변단어의 크기를 몇개로 할지를 **window size** 라고 한다. window size를 N으로 하면 중심단어 **좌우로 N개**가 주변단어가 된다. <br>
> 

### CBOW (Continuous Bag of Words)
- CBOW 모델은 주변 단어들로부터 중심 단어를 예측하는 방식

#### Dataset 구성
- CBOW/Skip-gram 모델 학습 데이터셋은 모두 token들이 one-hot encoding 되 있어야 한다.
  - **one hot vector**
    
    <img src="figures/wordwvec_dataset_onehotvector.png">
- Window size를 설정 한 뒤 문장을 중심 단어를 뒤로 이동 시키면서 중심단어와 주변단어들을 추출해 데이터셋을 만든다.
    > - 지정한 개수(window) 만큼씩 이동하면서 어떤 작업을 진행하는 것을 **Sliding window 방식** 이라고 한다.
- <span style="font-size:1.2em;font-weight:bold">window size 를 2로 설정한 뒤 데이터셋 구성 예</span>
    - **sliding window**
      
       <img src="figures/word2vec_cbow_sliding_window.png" width="700">


#### 모델 구조
**주변단어들의 one-hot vector를 Input, 중심단어의 one-hot vector를 output**으로 하여 CBOW 모델을 학습한다.   
- 예)
  - **vocab size(총단어수)**: 10,000
  - **window size**: 2
  - **embedding 차원**: 100 - 한 개 단어에 대해서 사이즈가 100인 일차원 벡터로 구성한다 = **단어별로 feature 100개씩 찾겠다**
  - **입력**: 주변단어(one-hot vector) - 4개 (window size x 2)
  - **출력**: 전체 단어들에 대해 중심 단어일 확률 출력(**다중분류 문제**) - 출력 결과가 만 개의 값으로 나오게 된다.
   
<img src="figures/word2vec_cbow.png" width="800">

- 입력으로 주변 단어를 받는다. -> 토큰들을 하나씩 넣는다. x1, x2, x3, x4를 따로따로 순서대로 넣는다. 
- 원핫 인코딩 -> input의 벡터 차원은 10000개 (단어 클래스가 10000개니까 원핫인코딩 하면 만개)

입력: `nn.Linear(10000, 100)` 이런 식으로

x1, .., x4를 차례로 입력해서, weight와 내적을 해서 나온 값을 평균을 내겠다! <- hidden>

input: 1 x 10000 (10000: class 수)

win: 10000 x 100 (100: embedding 차원)

wout: 100 x 10000 (100: embedding 차원, 10000: 클래스 수 - 출력 만 개 해야하니까)

nn.Linear - nn.activation func(ex. relu) - nn.Linear - ... - softmax

준지도 학습

결과: __ __ y __ __ -> y가 될 수 있는 단어의 확률을 계산 -> 역전파를 하면서 모델 학습을 하면 파라미터가 조정되면서, 중심 단어 y를 더 잘 맞추는 모델이 되는 거임!!

하이퍼파라미터: 임베딩 벡터 차원

- 입력 단어는 **one-hot vector** 다. 그래서 단어 index만 1이고 나머진 모두 0으로 구성된다.
이  입력 one-hot vector와 가중치 행렬 $W_{in}$이 가중합(행렬곱)을 계산하면 가중치 행렬에서 그 단어 index의 행(one-hot vector의 1의 index의 행)  행을 가져오는 것이 된다. 그래서 word2vec의 hidden layer를 계산하는 작업($X \cdot W_{in}$)은 **가중치 행렬 $W_{in}$에서 해당 단어에 해당하는 행을 찾는(lookup) 작업**을 하는 것이 된다.
    - ex)
\begin{align} 
\left[
\begin{matrix}
    0 & 0& 1 & 0
\end{matrix}
\right] \cdot \left[
\begin{matrix}
0.1 & 0.1 & 0.1 \\
0.2 & 0.2 & 0.2 \\
0.3 & 0.3 & 0.3 \\
0.4 & 0.4 & 0.4
\end{matrix}
\right] = \left[
\begin{matrix}
0.3 & 0.3 & 0.3
\end{matrix}
\right] \\
\text{행: 단어(4), 열: embedding 차원(3)}
\end{align}    
- Word2Vec의 학습은 **가중치 행렬의 각 행들이 단어들의 word embedding vector**가 되도록 **주변단어와 중심단어의 관계로 학습**하는 것이다.
- 학습이 완료 되면 $W_{in}$ 이나 $W_{out}$ 파라미터를 word embeding vector로 사용한다.




### Skip-gram

중심단어를 이용해 그 주변단어를 예측하는 모델을 구성한다.

#### Dataset 구성
- CBOW 모델과 마찬가지로 모든 token 들은 one-hot vector로 구성한다.
- **sliding window** 방식으로 이동하면서 **중심단어와 그 주변단어 쌍**으로 Dataset을 구성한다.
    - window size를 n으로 지정하면 중심단어를 input 그 중심단의 앞의 n개, 뒤의 n가 단어를 output으로 구성한다.
- <span style="font-size:1.2em;font-weight:bold">window size 를 2로 설정한 뒤 데이터셋 구성 예</span> 

  <img src="figures/wore2vec_skipgram_sliding_window.png" width="700">

주변 단어 4개를 한 번에 맞추는 게 아니라, 하나씩 맞춘다~!! 위는 cat과 sat으로 8개의 데이터셋이 만들어짐. 

#### 모델 구조
<img src='figures/word2vec_skipgram.png' width="800">

학습은 y1, y2, y3, y4 **따로따로**

- 모델의 입력으로 중심단어의 one-hot vector가 들어가고 모델을 주변단어를 추론한다. window size 가 2라면 총 4개의 주변단어를 추론한다.
- 학습 할 때는 각 주변단어들에 대한 개별 loss를 계산하고 그 합계를 최종 loss하여 $W_{in}$과 $W_{out}$ 을 update 한다.

### CBOW vs Skip-gram

- 여러 논문들에서 두 방식에 대한 성능 비교를 한 결과를 보면 대체적으로 **Skip-gram** 이 더 좋다.
    - 특히 말뭉치(Corpus)가 클수록 저빈도 단어나 유추 문제의 성능이 skip-gram이 더 뛰어난 경향이 있다.
    - skip-gram은 하나의 단어를 이용해 그 주변단어 여러개를 예측하는 것으로 주변단어로 중심단어 하나를 찾는 것(CBOW)보다 어려운 문제이다. Skip-gram이 CBOW 보다 더 어려운 상황에서 훈련하는 모델인 만큼 단어의 분산표현이 더 뛰어날 가능성이 커진다.

# Fastext
- Word2Vec 기반의 word embedding 방식. 모델 구조는 word2vec과 동일하다.
- Word2Vec은 단어(word) 단위로 학습을 한는데 비해 **Fastext**는 단어를 subword로 더 나눠 학습한다.

## 장점
- Subword 방식으로 단어를 더 쪼개서 학습하기 때문에 OOV(Out of Vocabulary) 에 대해 강점을 가진다.